<a href="https://colab.research.google.com/github/vedanshipathak/Mental-Health-Support-Chatbot/blob/main/Final_Model_mental_health_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb openai flask datasets evaluate torch transformers accelerate faiss-cpu


In [2]:
import pandas as pd
import re
# Load the dataset into a pandas DataFrame
df = pd.read_csv(r'/content/counselchat-data.csv')

# Function to strip HTML tags (like <p>, <br>, etc.)
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str(text))

# Apply cleaning to the 'answerText' column
df['clean_answer'] = df['answerText'].apply(remove_html_tags)

# Optional: Combine questionTitle and questionText for better context
df['full_question'] = df['questionTitle'].fillna('') + ' ' + df['questionText'].fillna('')

# Preview the cleaned data
df[['full_question', 'clean_answer']].head()
# Display first few rows
print(df.head())


                 questionID                                     questionTitle  \
0  5566fab2a64752d71ec3ca69  Escalating disagreements between mother and wife   
1  5566f94fa64752d71ec3ca64          I'm addicted to smoking. How can I stop?   
2  5567d26887a1cc0c3f3d8f46                    Keeping secrets from my family   
3  556bed15c969ba5861709df5         The Underlying Causes of Being Possessive   
4  556ba115c969ba5861709de6         Can I control anxiety without medication?   

                                        questionText  \
0  My wife and mother are having tense disagreeme...   
1  I'm planning to have baby, so I have to quit s...   
2  I have secrets in my mind, and I don't know wh...   
3  I am extremely possessive in my relationships ...   
4  I had a head injury a few years ago and my min...   

                                         questionUrl  \
0  https://counselchat.com/questions/escalating-d...   
1  https://counselchat.com/questions/i-m-addicted...   
2  https

In [3]:
# Display column names
print("Column Names:", df.columns.tolist())


Column Names: ['questionID', 'questionTitle', 'questionText', 'questionUrl', 'topics', 'therapistName', 'therapistUrl', 'answerText', 'upvotes', 'clean_answer', 'full_question']


In [4]:
import html

# Apply to the cleaned answer column
df['decoded_answer'] = df['clean_answer'].apply(html.unescape)

# Optional: also decode the question column if needed
df['decoded_question'] = df['full_question'].apply(html.unescape)

# Preview
df[['decoded_question', 'decoded_answer']].head()

,decoded_question,decoded_answer
0,Escalating disagreements between mother and wi...,What you are describing is something psycholog...
1,I'm addicted to smoking. How can I stop? I'm p...,Hi. Good for you in planning ahead to do what'...
2,Keeping secrets from my family I have secrets ...,It sounds like keeping the secrets has become ...
3,The Underlying Causes of Being Possessive I am...,Hi there. It's great you are able to realize t...
4,Can I control anxiety without medication? I ha...,You didn't say what or how many medications yo...


In [5]:
# Extract relevant columns
questions = df["decoded_question"].dropna().tolist()  # Remove NaN values if any
answers = df["decoded_answer"].dropna().tolist()

# Print a sample
print("Sample Question:", questions[0])
print("Sample Answer:", answers[0])

Sample Question: Escalating disagreements between mother and wife My wife and mother are having tense disagreements. In the past, they’ve had minor differences. For example, my wife would complain to me my mother is too overbearing; my mother would complain my wife is lazy.
However, it’s intensified lately. I think the cause is my wife talked back to her once. Now, any little disagreement is magnified, leading to major disagreements. What can I do?
Sample Answer: What you are describing is something psychologists have termed "triangulation" which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been "triangulated" by your wife and mother.This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, es

In [6]:
!pip install langchain openai


In [7]:
from langchain.prompts import PromptTemplate

# General Mental Health Support Prompt (already defined)
general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Anxiety-specific Prompt
anxiety_prompt = PromptTemplate(
    template="""
You are an empathetic therapist specialized in anxiety management. Provide calm, supportive, and actionable advice specifically targeted towards managing anxiety. Suggest practical techniques like mindfulness, breathing exercises, or grounding practices.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Depression-specific Prompt
depression_prompt = PromptTemplate(
    template="""
You are a compassionate therapist specialized in supporting clients experiencing sadness, depression, or feelings of hopelessness. Offer empathy, reassurance, and practical suggestions such as self-care activities, reaching out for support, or considering therapy.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Relationship-specific Prompt
relationship_prompt = PromptTemplate(
    template="""
You are a caring therapist specializing in relationship and family counseling. Provide empathetic and practical advice on communication strategies, understanding perspectives, conflict resolution, or improving interpersonal relationships.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)


# Example usage:
client_message = "I've been feeling very anxious lately and don't know how to cope."
formatted_prompt = general_prompt.format(client_message=client_message)
print(formatted_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I've been feeling very anxious lately and don't know how to cope.
Therapist:



In [8]:
def get_prompt(client_message):
    if any(word in client_message.lower() for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

# Example Test
test_message = "I'm having serious conflicts with my wife, and I feel stuck."
selected_prompt = get_prompt(test_message)
print(selected_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I'm having serious conflicts with my wife, and I feel stuck.
Therapist:



In [9]:
!pip install transformers torch accelerate


In [10]:
from huggingface_hub import login
login()


In [11]:
import requests

try:
    response = requests.get("https://huggingface.co", timeout=5)
    if response.status_code == 200:
        print(" Internet is working. Hugging Face is reachable!")
    else:
        print(" Internet issue: Received status code", response.status_code)
except requests.ConnectionError:
    print(" No internet connection detected!")


 Internet is working. Hugging Face is reachable!


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#  Load tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

#  Fix: Set padding token to avoid ValueError when padding
tokenizer.pad_token = tokenizer.eos_token

#  Load model with half precision and auto device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded successfully!")



`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully!


In [15]:
def generate_response(client_message):
    # Format the structured prompt
    prompt = (
    "You are a compassionate therapist. Respond with empathy, support, and actionable advice. "
    "Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.\n\n"
    f"Client: {client_message}\nTherapist:"
)


    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt",padding=True).to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9,pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id)

    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

#  Test the chatbot
test_message = "I've been feeling very anxious lately and don't know how to cope."
response = generate_response(test_message)
print("\nChatbot Response:\n", response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Chatbot Response:
 You are a compassionate therapist. Respond with empathy, support, and actionable advice. Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I've been feeling very anxious lately and don't know how to cope.
Therapist: I'm sorry to hear that you're going through this. Anxiety can be really challenging to deal with, but there are many things you can do to help manage it. Have you tried any relaxation techniques, such as deep breathing or meditation? These can help calm your mind and reduce feelings of anxiety. It might also be helpful to talk to a therapist who can guide you through strategies for managing anxiety and provide support as you work through it. In the meantime, try to prioritize self-care activities that bring you joy and help you feel relaxed, like taking a walk in nature, reading a book, or practicing yoga


In [16]:
!pip install datasets evaluate sacrebleu nltk


In [17]:
!pip install rouge_score

In [18]:
import evaluate
import math
import torch
import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer

nltk.download("punkt")  # Needed for tokenization in BLEU & ROUGE

# Load BLEU and ROUGE metrics from Hugging Face's Evaluate library
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Function to calculate perplexity
def calculate_perplexity(response):
    inputs = tokenizer(response, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = math.exp(loss.item())
    return perplexity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
def evaluate_response(generated_response, reference_response):
    # Compute BLEU score (expects full sentences, not tokenized words)
    bleu_score = bleu.compute(predictions=[generated_response], references=[[reference_response]])

    # Compute ROUGE score (expects full sentences)
    rouge_score = rouge.compute(predictions=[generated_response], references=[reference_response])

    # Compute perplexity
    perplexity = calculate_perplexity(generated_response)

    return {
        "BLEU Score": bleu_score["bleu"],
        "ROUGE Score": rouge_score["rouge1"],
        "Perplexity": perplexity
    }


In [20]:
 import nltk
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [21]:
# Example chatbot response vs reference therapist response
generated_response = """
I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment.
"""
reference_response = """
I hear that you're feeling anxious, and that's completely understandable. A helpful way to manage anxiety is practicing deep breathing exercises or grounding techniques.
Taking a moment to focus on your breath or journaling can also be beneficial. Would you like to explore more coping strategies together?
"""

# Run evaluation
scores = evaluate_response(generated_response, reference_response)
print(scores)


{'BLEU Score': 0.12811300935224262, 'ROUGE Score': np.float64(0.38834951456310685), 'Perplexity': 2.641627001339111}


In [22]:
!pip install -U sentence-transformers



In [23]:
!pip install faiss-cpu langchain chromadb


In [24]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a free embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Free alternative to OpenAIEmbeddings

# Example dataset
conversation_texts = [
    "How do I deal with anxiety?",
    "What should I do if I'm feeling very anxious?",
    "How can I manage stress effectively?",
    "I've been feeling overwhelmed at work, how do I handle it?"
]

# Convert questions to vector embeddings
vectors = embedding_model.encode(conversation_texts, convert_to_numpy=True)

# Create FAISS index
vector_array = np.array(vectors, dtype=np.float32)
faiss_index = faiss.IndexFlatL2(vector_array.shape[1])
faiss_index.add(vector_array)
print("FAISS Index Created Successfully with SentenceTransformers!")



FAISS Index Created Successfully with SentenceTransformers!


In [25]:
def retrieve_context(query, top_k=3):
    query_vector = np.array([embedding_model.encode(query)], dtype=np.float32)
    distances, indices = faiss_index.search(query_vector, k=top_k)

    # Fetch similar questions and associated answers explicitly
    similar_q_and_a = [
        (conversation_texts[i], df['decoded_answer'].iloc[i])
        for i in indices[0]
    ]
    return similar_q_and_a



In [31]:

def generate_response_with_rag(client_message):
    # Retrieve similar past questions and answers
    context_qas = retrieve_context(client_message)[:5]  # top 5 similar QAs


    # Format the retrieved context for clarity
    formatted_context = "\n\n---\n\n".join([f"Q: {q}\nA: {a}" for q, a in context_qas])


    #  Enhanced prompt with clear instructions
    prompt = f"""
You are a compassionate therapist helping people with mental health concerns.

Below are similar past conversations that might help you understand the client's situation better and guide your response:

{formatted_context}

Based on the above context, now respond to the client below with empathy and actionable advice.
Your response should be emotionally supportive and, if appropriate, recommend mindfulness, self-care, or therapeutic strategies.

Client: {client_message}
Therapist:
"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024).to("cuda")


    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=250,
            temperature=0.6,
            top_p=0.95,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode and return just the therapist's message
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("Therapist:")[-1].strip()
    return response


In [32]:
test_messages = [
    "hello i am sad",
    "i feel overwhelmed by everything at work",
    "my mom and wife keep arguing and it’s stressing me out",
    "i can't sleep at night and keep overthinking"
]

for msg in test_messages:
    print("\nClient:", msg)
    print("Therapist:", generate_response_with_rag(msg))
    print(evaluate_response())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Client: hello i am sad
Therapist: You are a compassionate therapist helping people with mental health concerns.

Below are similar past conversations that might help you understand the client's situation better and guide your response:

Q: What should I do if I'm feeling very anxious?
A: Hi. Good for you in planning ahead to do what's healthiest for your baby (and yourself). That's a great first step! It's also good that you are able to identify that it's not always a physical need that's driving the addiction.For the next steps, I would suggest trying to figure out when the psychological cravings for a cigarette occur. The psychological (or mental) cravings are usually based out of habit, such as having a cigarette after a meal. And if you're consciously trying to quit, you'll find the craving starts with simply thinking about having a cigarette, then usually moves on to thinking about how good it made you feel, etc., etc. Well, if I'm on a diet and I continue to let myself think abo

TypeError: evaluate_response() missing 2 required positional arguments: 'generated_response' and 'reference_response'

In [33]:
evaluate_response

<function __main__.evaluate_response(generated_response, reference_response)>

In [34]:
# Example chatbot response vs reference therapist response
generated_response = """
I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment.
"""
reference_response = """
I hear that you're feeling anxious, and that's completely understandable. A helpful way to manage anxiety is practicing deep breathing exercises or grounding techniques.
Taking a moment to focus on your breath or journaling can also be beneficial. Would you like to explore more coping strategies together?
"""

# Run evaluation
scores = evaluate_response(generated_response, reference_response)
print(scores)


{'BLEU Score': 0.12811300935224262, 'ROUGE Score': np.float64(0.38834951456310685), 'Perplexity': 2.641627001339111}


Backend Setup


In [35]:

!pip install flask flask-cors --quiet
!pip install pyngrok --quiet
!pip install transformers accelerate --quiet


In [36]:
from pyngrok import ngrok
# Set your authtoken here (paste your token inside the quotes)
ngrok.set_auth_token("2pFUVPfPq4fWMfvhNuaaqMe9Soh_3Zt7xCBRZKEGEJdGNXnZm")


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

anxiety_prompt = PromptTemplate(
    template="""
You are a calm and reassuring mental health assistant. The user is feeling anxious or overwhelmed.
Offer calming techniques, grounding exercises, and positive affirmations.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

depression_prompt = PromptTemplate(
    template="""
You are a caring and understanding therapist. The user feels depressed, hopeless, or sad.
Listen supportively and offer encouragement, self-compassion practices, and gentle help.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

relationship_prompt = PromptTemplate(
    template="""
You are a relationship counselor. The user is experiencing issues in relationships or family.
Help them understand their emotions and suggest communication strategies.

Client: {client_message}
Therapist:""",
    input_variables=["client_message"],
)

def get_prompt(client_message):
    message = client_message.lower()
    if any(word in message for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in message for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in message for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

def run_model(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Therapist:" in text:
        text = text.split("Therapist:")[-1].strip()

    return text


app = Flask(__name__)
CORS(app)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.json
    user_input = data.get("message", "")
    prompt = get_prompt(user_input)
    response = run_model(prompt)
    return jsonify({"response": response})

port = 5001
public_url = ngrok.connect(port)
print(f"🚀 Public URL: {public_url}")
app.run(port=port)


🚀 Public URL: NgrokTunnel: "https://4b9fc6c56453.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 17:53:20] "OPTIONS /chat HTTP/1.1" 200 -
/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:2532: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 17:54:10] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 17:54:31] "OPTIONS /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Oct/2025 17:54:39] "POST /chat HTT